#### Here I will document my method for cleaning my data files, which were taken from Kaggle's dataset “Comments on articles published in the New York Times” (https://www.kaggle.com/aashita/nyt-comments).
    
This code is broken into two sections - one cleans the data files with articles and one cleans the data files with comments. Here I test my cleaning using only a single file, but by concatenating data files together, I can run this code on the entire dataset.

In [1]:
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allArticles.csv")
train = pd.read_csv(r"C:\Users\msteele9\Documents\Springboard\Springboard\Data\ArticlesApril2017.csv")

train.head(5)

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


# Initial goals: 

-Make sure the contents of each field are the correct type and have no missing data (i.e. scrub the 'NaN' from the 'abstract' field)

-Make sure that the data comes properly tokenized

-Convert all words to lowercase (to avoid confusion between uppercase and lowercase versions of the same word)

Several of these data columns (articleID, articleWordCount, multimedia, printPage) contain only integers or single lowercase words.

In [2]:
#nltk.download('wordnet')
#for column in train:
    #print(train[column].get_dtype_counts())
    
print(train.dtypes.value_counts())

print("")

for column in train:
    print(train[column].dtypes)

object    13
int64      3
dtype: int64

object
object
int64
object
object
object
object
int64
object
int64
object
object
object
object
object
object


###### From the above code, the only integer columns are 2, 7 and 9. The rest are string columns and need to be converted to lowercase. 

In [3]:
sampleSize = 4000000
train = pd.read_csv(r"C:\Users\msteele9\Documents\Springboard\Springboard\Data\ArticlesApril2017.csv", header=0, nrows=sampleSize)
#train.head(5)

wn = nltk.WordNetLemmatizer()

nonstrings = [2, 7, 9]


train= train.astype(str)
train.fillna(0)

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in nonstrings:
            doc[column] = doc[column].astype(str)
            continue
        doc[column] = doc[column].str.replace('[^\w\s]','')
        doc[column] = doc[column].str.lower()
        #doc[column] = doc[column].str.strip()
        doc[column] = doc[column].replace(np.nan, ' ', regex=True)
        doc[column].apply(nltk.word_tokenize)
        doc[column].apply(lemmatize_text)
        doc[column] = [token for token in doc[column] if token not in stop_words]     
    return doc

def lemmatize_text(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

clean_art = clean_articles(train)

clean_art.head(5)

#print('\n'.join(clean))


,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,nan,58def1347c459f24986d7c80,716,by stephen hiltner and susan lehman,article,finding an expansive view of a forgotten peop...,photography new york times niger ferguson adam...,3,insider,2,20170401 001541,unknown,one of the largest photo displays in times his...,the new york times,news,httpswwwnytimescom20170331insidernigermigrants...
1,nan,58def3237c459f24986d7c84,823,by gail collins,article,and now the dreaded trump curse,united states politics and government trump do...,3,oped,23,20170401 002358,unknown,meet the gang from under the bus,the new york times,oped,httpswwwnytimescom20170331opinionandnowthedrea...
2,nan,58def9f57c459f24986d7c90,575,by the editorial board,article,venezuelas descent into dictatorship,venezuela politics and government maduro nicolas,3,editorial,22,20170401 005306,unknown,a court ruling annulling the legislatures auth...,the new york times,editorial,httpswwwnytimescom20170331opinionvenezuelasdes...
3,nan,58defd317c459f24986d7c95,1374,by michael powell,article,stain permeates basketball blue blood,basketball college university of north carolin...,3,sports,1,20170401 010652,college basketball,for two decades until 2013 north carolina enga...,the new york times,news,httpswwwnytimescom20170331sportsncaabasketball...
4,nan,58df09b77c459f24986d7ca7,708,by deb amlen,article,taking things for granted,crossword puzzles,3,games,0,20170401 020014,unknown,in which howard barkin and will shortz teach u...,the new york times,news,httpswwwnytimescom20170331crosswordstakingthin...


###### I want to check for cells that have missing elements.

In [4]:
print("Any null values left: "),
clean_art.isnull().values.any()

Any null values left: 


False

###### Now we want a separate section to clean the comment files.

In [5]:
#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allComments.csv")

sampleSize = 10000
train = pd.read_csv(r"C:\Users\msteele9\Documents\Springboard\CommentsApril2017.csv", nrows=sampleSize)

train= train.astype(str)
train.fillna(0)
strings = [1, 5, 10, 18, 22, 24, 25, 26, 29, 30, 33]

def lemmatize_text(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in strings:         
            doc[column] = doc[column].str.replace('[^\w\s]','')
            doc[column] = doc[column].str.lower()
            #doc[column] = doc[column].str.strip()
            doc[column] = doc[column].replace(np.nan, '', regex=True)
            doc[column].apply(nltk.word_tokenize)
            doc[column].apply(lemmatize_text)
            doc[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        else:
            doc[column] = doc[column].astype(str)
            continue
    return doc

clean_comments = clean_articles(train)
#The second command takes awhile to run
clean_comments.head(5)

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,this project makes me happy to be a 30 year ti...,22022598.0,22022598,<br/>,comment,1491237056.0,1,False,0.0,...,riverside ca,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
1,1491188619,stunning photos and reportage infuriating that...,22017350.0,22017350,nan,comment,1491180489.0,1,False,0.0,...,br,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
2,1491188617,brilliant work from conception to execution iv...,22017334.0,22017334,<br/>,comment,1491179470.0,1,False,0.0,...,raleigh nc,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
3,1491167820,nyt reporters should provide a contributors li...,22015913.0,22015913,<br/>,comment,1491150196.0,1,False,0.0,...,missouri usa,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
4,1491167815,could only have been done in print stunning,22015466.0,22015466,<br/>,comment,1491147284.0,1,False,0.0,...,tucson arizona,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news


In [6]:
print("Any null values left: "), print(clean_comments.isnull().values.any())

Any null values left: 
False


(None, None)

In [7]:
clean_comments.nunique()

approveDate               5073
commentBody               9994
commentID                10000
commentSequence          10000
commentTitle                 2
commentType                  3
createDate                9715
depth                        3
editorsSelection             2
parentID                  1646
parentUserDisplayName     1302
permID                   10000
picURL                     781
recommendations            358
recommendedFlag              1
replyCount                  24
reportAbuseFlag              1
sharing                      2
status                       1
timespeople                  1
trusted                      2
updateDate                5380
userDisplayName           5118
userID                    6293
userLocation              2242
userTitle                    2
userURL                      2
inReplyTo                 1646
articleID                   37
sectionName                  8
newDesk                     12
articleWordCount            37
printPag

I see that there are no null values remaining, but looking at the dataframe I see that several columns contain nothing but 'nan' strings or otherwise have only one value. I want to drop the commentTitle (contains only <br/> or nan), recommendedFlag, reportAbuseFlag, status, timespeople, userTitle and userURL columns.

In [8]:
clean_comments.drop(columns=['commentTitle', 'recommendedFlag', 'reportAbuseFlag', 'status', 'timespeople', 'userTitle', 'userURL'], axis=1, inplace=True)

clean_comments.head(10)

,approveDate,commentBody,commentID,commentSequence,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,...,userDisplayName,userID,userLocation,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,this project makes me happy to be a 30 year ti...,22022598.0,22022598,comment,1491237056.0,1,False,0.0,nan,...,rob gayle,46006296,riverside ca,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
1,1491188619,stunning photos and reportage infuriating that...,22017350.0,22017350,comment,1491180489.0,1,False,0.0,nan,...,susan a,29202761,br,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
2,1491188617,brilliant work from conception to execution iv...,22017334.0,22017334,comment,1491179470.0,1,False,0.0,nan,...,meta,63944806,raleigh nc,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
3,1491167820,nyt reporters should provide a contributors li...,22015913.0,22015913,comment,1491150196.0,1,False,0.0,nan,...,tom wyrick,1266184,missouri usa,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
4,1491167815,could only have been done in print stunning,22015466.0,22015466,comment,1491147284.0,1,False,0.0,nan,...,joe sharkey,61121360,tucson arizona,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
5,1491142576,thank you new york times people should be supp...,22012085.0,22012085,comment,1491128692.0,1,False,0.0,nan,...,dramaman,59125002,new york,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
6,1491060909,proof that photojournalism is alive and well e...,22003784.0,22003784,comment,1491056012.0,1,False,0.0,nan,...,bob williams,320791,east northport ny,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
7,1491252241,the oasis initiative which i started with prof...,22024897.0,22024897,userreply,1491247899.0,2,False,22015913.0,tom wyrick,...,alisha g,65289571,tahoma ca,22015913,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
8,1491668599,i agree ive just spent 30 minutes trying to fi...,22082978.0,22082978,userreply,1491665056.0,2,False,22015913.0,tom wyrick,...,kathy mortensen,76442479,ann arbor,22015913,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
9,1491064414,how about katrina pierson back to palookaville...,22004930.0,22004930,comment,1491061051.0,1,False,0.0,nan,...,gaucho54,59237391,california,0,58def3237c459f24986d7c84,unknown,oped,823.0,23,oped


###### I check my working directory to make sure I am saving the files where I want them stored.

In [9]:
# 
import os
os.chdir(r'C:\Users\msteele9\Documents\Springboard\Springboard\Data')
art_file_name = "cleaned_article_data.csv"
clean_art_csv = clean_art.to_csv(art_file_name, encoding='utf-8', index=False)

In [10]:
com_file_name = "cleaned_comment_data.csv"
clean_com_csv = clean_comments.to_csv(com_file_name, encoding='utf-8', index=False)

###### Here I have saved the cleaned data files to storage. This is a good break point where I can use these files to begin looking at possible ML models. 
    
   In the next section, I will explore the data, looking at features and ways to visualize the feature set. I begin by re-importing the libraries that I will use. That way, this section of the notebook can be reran separately later. I will load the cleaned files in so that I know I am not making any changes to the stored version.

In [11]:
import pandas as pd
import numpy as np
import nltk
#ntlk.download()
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

file_path_art = r'C:\Users\msteele9\Documents\Springboard\Springboard\Data\cleaned_article_data.csv'
clean_art = pd.read_csv(file_path_art, index_col = False)
clean_art

file_path_comments = r'C:\Users\msteele9\Documents\Springboard\Springboard\Data\cleaned_comment_data.csv'
clean_comments = pd.read_csv(file_path_comments, index_col = False)

###### In the above cell, I loaded in the cleaned article data. Here I load in the cleaned data and do preprocessing to run the data through a random forest model. 
    
   I want to make sure that my data can actually be loaded and that my simple test will produce results of some sort - this way I know that my data will not break when I try to load it.
   
   My target is the number of recommendations that the comment receives. I want to know how well a simple model can predict the number of recommendations a comment will receive when given the rest of that comment's data.


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

features = clean_comments.columns.tolist()
output = 'recommendations'
features.remove('recommendations')

for column in clean_comments.columns:
    clean_comments[column] = clean_comments[column].astype(str)
    if clean_comments[column].dtype == type(object):
        clean_comments[column] = le.fit_transform(clean_comments[column])

#print(features)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

clean_comments.astype(float)
clean_comments.head(5)

#clean_comments.dtypes

,approveDate,commentBody,commentID,commentSequence,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,permID,picURL,recommendations,replyCount,sharing,trusted,updateDate,userDisplayName,userID,userLocation,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,4183,8195,8329,8329,0,8081,0,0,0,887,8671,17,110,0,0,0,4385,4112,1801,1639,0,0,7,4,29,4,1
1,3457,6605,7166,7166,0,6940,0,0,0,887,7306,17,1,0,0,0,3648,4600,975,267,0,0,7,4,29,4,1
2,3456,1089,7160,7160,0,6934,0,0,0,887,7301,17,167,0,0,0,3647,3254,3698,1603,0,0,7,4,29,4,1
3,3292,5470,6606,6606,0,6388,0,0,0,887,6844,17,298,11,0,0,3473,4795,108,1187,0,0,7,4,29,4,1
4,3291,1347,6385,6385,0,6176,0,0,0,887,6688,14,243,0,0,0,3472,2341,3302,2016,0,0,7,4,29,4,1


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()

param = {'n_estimators': [5, 10, 25],
         'max_depth': [5, 10, 20, 50] }

gs=GridSearchCV(rf, param, cv=5, n_jobs=5)

gs_fit = gs.fit(clean_comments[features], clean_comments[output])


C:\Users\msteele9\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\msteele9\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [14]:
pd.DataFrame(gs_fit.cv_results_).sort_values(by=['rank_test_score']).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.242198,0.047237,0.048000,0.011983,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.109312,0.100149,0.046500,0.111111,0.223754,0.1171,0.056561,1
2,0.583198,0.106234,0.128599,0.037398,5,25,"{'max_depth': 5, 'n_estimators': 25}",0.097166,0.105107,0.049566,0.107981,0.223754,0.1154,0.056288,2
0,0.115798,0.020683,0.028802,0.007985,5,5,"{'max_depth': 5, 'n_estimators': 5}",0.084570,0.090729,0.067961,0.117893,0.221103,0.1147,0.053688,3
4,0.446998,0.079622,0.038001,0.007293,10,10,"{'max_depth': 10, 'n_estimators': 10}",0.052182,0.083292,0.015841,0.094418,0.220573,0.0912,0.067978,4
5,1.137200,0.238564,0.113599,0.034302,10,25,"{'max_depth': 10, 'n_estimators': 25}",0.062978,0.074368,0.016863,0.094418,0.190880,0.0864,0.056290,5


###### A quick GridSearch with a random forest classifier was able run on my data set and produce an accuracy score. My goal was to confirm that I can run ML algorithms on my data set and get sensible results; this test seems to verify this.

Here I am looking to see what the best model is doing. Only a few of our features have a strong impact on the number of recommendations. It looks like the dominant features are the approve date and the comment body. These intuitively make sense - we expect the content of the comment to be one of if not the most important feature, and the time the comment is posted likely determines how many people will see it.

In [15]:
import matplotlib.pyplot as plt

importances = gs_fit.best_estimator_.feature_importances_

std = np.std([tree.feature_importances_ for tree in gs_fit.best_estimator_.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(clean_comments[features].shape[1]):
    print("%d. feature %d : (%s) (%f)" % (f + 1, indices[f], features[f], importances[indices[f]]))
    
    # Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(clean_comments[features].shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(clean_comments[features].shape[1]), indices)
plt.xlim([-1, clean_comments[features].shape[1]])
plt.show()

Feature ranking:
1. feature 20 : (approveDate) (0.130939)
2. feature 8 : (commentBody) (0.114369)
3. feature 22 : (commentID) (0.091900)
4. feature 17 : (commentSequence) (0.089533)
5. feature 15 : (commentType) (0.069414)
6. feature 23 : (createDate) (0.065152)
7. feature 24 : (depth) (0.059461)
8. feature 25 : (editorsSelection) (0.055708)
9. feature 10 : (parentID) (0.046112)
10. feature 19 : (parentUserDisplayName) (0.044994)
11. feature 21 : (permID) (0.039039)
12. feature 3 : (picURL) (0.034184)
13. feature 0 : (replyCount) (0.034116)
14. feature 9 : (sharing) (0.024503)
15. feature 4 : (trusted) (0.018952)
16. feature 5 : (updateDate) (0.017675)
17. feature 12 : (userDisplayName) (0.016609)
18. feature 2 : (userID) (0.013633)
19. feature 16 : (userLocation) (0.013002)
20. feature 1 : (inReplyTo) (0.006463)
21. feature 18 : (articleID) (0.005166)
22. feature 11 : (sectionName) (0.004056)
23. feature 6 : (newDesk) (0.003892)
24. feature 14 : (articleWordCount) (0.001128)
25. featu

<Figure size 640x480 with 1 Axes>